In [1]:
import math
import pandas as pd
import json

### Cálculo de Entropia

In [2]:
# realiza o cálculo da entropia
def entropy(p):
    
    # se o valor for maio que zero, é possível computar o log
    if (p > 0):
        return p * math.log2(p)
    
    else:
        return 0
    
# calcula a entropia do dataset
def dataset_entropy(df):
    
    h = 0 # entropia
    
    # para cada classe do dataset, calcula a entropia e realiza a soma
    for item in df['class'].unique():
        
        # calcula a entropia para cada classe e realiza o somatório em h
        h += - entropy(df.loc[df['class'] == item].shape[0] / len(df))
        
    return h 

# calcula a entropia para de determinado atributo
def attribute_entropy(df, attribute, labels):

    # separa os valores distintos existentes no atributo
    att_values = df[attribute].unique()
    
    attrib_entropy = 0
    
    # para cada valor distinto do atributo
    for t in att_values:
        
        subset_entropy = 0
        
        # seleciona os valores do atributo iguais ao valor do iterador
        subset = df.loc[df[attribute] == t]
        
        # proporção entre o subset e o dataset
        p = subset.shape[0] / len(df)
        
        # para cada classe, calcula a entropia em relação ao subset e armanena o somatório
        for item in labels:
            subset_entropy += - entropy(subset.loc[subset['class'] == item].shape[0] / subset.shape[0])
        
        # a entropia do atributo é a soma da entropia de todos os subsets multiplicado pela variável p
        attrib_entropy += subset_entropy * p
        
    return attrib_entropy

### Ganho de Informação

In [3]:
# calcula o ganho de informação
def information_gain(df, attributes, labels):
    
    # armazena a entropia do dataset
    h = dataset_entropy(df)
    
    # dicionário para armazenar os valores do ganho
    gain_dict = {}
    
    # para cada atributo, calcula-se sua entropia
    for att in attributes:
        gain_dict[att] = h - attribute_entropy(df, att, labels)
        
    return gain_dict

### Algoritmo ID3 

In [4]:
def id3_tree(df, attributes, labels):
    
    # cria um dicionário para a árvore
    tree = {}
    
    # se houver apenas uma classe distinta na base, retornar a própria classe
    if (len(df['class'].unique().tolist()) == 1):
        tree['class'] = df['class'].iloc[0]
        return tree
    
    # se não houver atributos, retornar a classe mais comum na base
    if (len(attributes) == 0):
        tree['class'] = df['class'].value_counts().idxmax()
        return tree
    
    else:
         
        # seleciona o atributo que tem o maior ganho
        dict_gain = information_gain(df, attributes, labels)
        max_gain = max(dict_gain, key=dict_gain.get)
        
        # captura os valores distintos do atributo com maior ganho
        values = df[max_gain].unique()
        
        # adiciona na raiz o atributo com maior ganho
        tree[max_gain] = {}
        
        # retira o atributo atual
        attributes.remove(max_gain)
        
        for v in values:

            # cria um subset com os valores do atributo com o maior ganho que seja igual aos valores distintos
            subset = df[df[max_gain] == v]
            
            # adiciona nó e chama recursivamente o algoritmo para construção da árvore
            tree[max_gain][v] = id3_tree(subset, attributes, labels)
            
    return tree

### Definições da Função Principal

In [5]:
def main():
    
    # leitura arquivo do dataset
    df = pd.read_csv('data/iris.data', header = None, encoding='utf-8')
    
    # definição de colunas
    df.columns = ['sepal length in cm', 'sepal width in cm', 'petal length in cm', 'petal width in cm' , 'class']
    
    # lista de características do dataset
    attributes = ['sepal length in cm', 'sepal width in cm', 'petal length in cm', 'petal width in cm']

    # lista de classes
    labels = df['class'].unique().tolist()
    
    # cálculo da entropia do dataset
    print('\ndataset entropy: ' + str(dataset_entropy(df)))
    
    # cálculo do ganho de informação
    dict_gain = information_gain(df, attributes, labels)

    # imprime atributo com o maior ganho do dataset
    print('\natributo com maior ganho de informação: ' + str(max(dict_gain, key=dict_gain.get)))
    
    # executa id3
    id3 = id3_tree(df, attributes, labels)
    
    # imprime árvore
    print ('\nÁrvore ID3\n' + json.dumps(id3, indent=1))
    
if __name__ == "__main__": main()


dataset entropy: 1.584962500721156

atributo com maior ganho de informação: petal length in cm

Árvore ID3
{
 "petal length in cm": {
  "1.4": {
   "class": "Iris-setosa"
  },
  "1.3": {
   "class": "Iris-setosa"
  },
  "1.5": {
   "class": "Iris-setosa"
  },
  "1.7": {
   "class": "Iris-setosa"
  },
  "1.6": {
   "class": "Iris-setosa"
  },
  "1.1": {
   "class": "Iris-setosa"
  },
  "1.2": {
   "class": "Iris-setosa"
  },
  "1.0": {
   "class": "Iris-setosa"
  },
  "1.9": {
   "class": "Iris-setosa"
  },
  "4.7": {
   "class": "Iris-versicolor"
  },
  "4.5": {
   "sepal length in cm": {
    "6.4": {
     "class": "Iris-versicolor"
    },
    "5.7": {
     "class": "Iris-versicolor"
    },
    "5.6": {
     "class": "Iris-versicolor"
    },
    "6.2": {
     "class": "Iris-versicolor"
    },
    "6.0": {
     "class": "Iris-versicolor"
    },
    "5.4": {
     "class": "Iris-versicolor"
    },
    "4.9": {
     "class": "Iris-virginica"
    }
   }
  },
  "4.9": {
   "sepal width in c